<h1 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4); 
           color: white; 
           padding: 20px; 
           border-radius: 10px; 
           text-align: center; 
           font-family: Arial, sans-serif; 
           text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Multimodal RAG with Amazon Bedrock
</h1>

This notebook demonstrates how to implement a multi-modal Retrieval-Augmented Generation (RAG) system using Amazon Bedrock. Many documents contain a mixture of content types, including text and images. Traditional RAG applications often lose valuable information captured in images. With the emergence of Multimodal Large Language Models (MLLMs), we can now leverage both text and image data in our RAG systems.

In this notebook, we'll explore one approach to multi-modal RAG (`Option 1`):

1. Use multimodal embeddings (such as Amazon Titan) to embed both images and text
2. Retrieve relevant information using similarity search
3. Pass raw images and text chunks to a multimodal LLM for answer synthesis using Claude 3/3.5 Multimodal

We'll use the following tools and technologies:

- [LangChain](https://python.langchain.com/v0.2/docs/introduction/) to build a multimodal RAG system
- [faiss](https://github.com/facebookresearch/faiss) for similarity search
- [Claude 3/3.5 Multimodal](https://www.anthropic.com/news/claude-3-5-sonnet) for answer synthesis
- [Amazon Titan Multimodal Embeddings](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-multiemb-models.html) for image embeddings
- [Amazon Bedrock](https://aws.amazon.com/bedrock/) for accessing powerful AI models, like the ones above
- [pymupdf](https://pymupdf.readthedocs.io/en/latest/) to parse images, text, and tables from documents (PDFs)
- [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) for interacting with Amazon Bedrock

This approach allows us to create a more comprehensive RAG system that can understand and utilize both textual and visual information from our documents.

## Prerequisites

Before running this notebook, ensure you have the following packages and dependencies installed:

- Python 3.9 or later
- langchain
- boto3
- faiss
- pymupdf
- tabula
- tesseract
- requests

Let's get started with building our multi-modal RAG system using Amazon Bedrock!

![Multimodal RAG with Amazon Bedrock](../img/multimodal-rag1.png)

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Importing the libs
</h2>

In [ ]:
#!pip install jpype1 tabula-py PyMuPDF

In [ ]:
import boto3
import tabula
import faiss
import json
import base64
import pymupdf
import requests
import os
import logging
import numpy as np
from tqdm import tqdm
from botocore.exceptions import ClientError
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython import display


logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)


<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Data Loading
</h2>

In [ ]:
# Downloading the dataset - URL of the "Attention Is All You Need" paper (Replace it with the URL of the PDF file/dataset you want to download)
url = "https://arxiv.org/pdf/1706.03762.pdf"

# Set the filename and filepath
filename = "attention_paper.pdf"
filepath = os.path.join("data", filename)

# Create the data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(filepath, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully: {filepath}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
# Display the PDF file
display.IFrame(filepath, width=1000, height=600)

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
   Data Extraction
</h2>

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
doc = pymupdf.open(filepath)
num_pages = len(doc)

# Define the directories to store the extracted text, images and page images from each page
image_save_dir = "data/processed_images"
text_save_dir = "data/processed_text"
table_save_dir = "data/processed_tables"
page_images_save_dir = "data/processed_page_images"

# Chunk the text for effective retrieval
chunk_size = 700
overlap= 200

# Process chunks with LangChain's RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
                                                chunk_size=chunk_size,
                                                chunk_overlap=overlap,
                                                length_function=len,
                                              )

# Process all pages of the PDF
items = []
for page_num in tqdm(range(num_pages), desc="Processing PDF pages"):
    page = doc[page_num]
    text = page.get_text()

    # Step 1: Get/extract all TABLES in the curremt page and store 
    try:
        tables = tabula.read_pdf(filepath, pages=page_num + 1, multiple_tables=True)

        if tables:
            for table_idx, table in enumerate(tables):
                # Convert the table DataFrame to a string format (Markdown-style for clarity)
                table_text = "\n".join([" | ".join(map(str, row)) for row in table.values])

                # Save the table text as a file
                table_file_name = f"{table_save_dir}/{os.path.basename(filepath)}_table_{page_num}_{table_idx}.txt"
                os.makedirs(table_save_dir, exist_ok=True)
                with open(table_file_name, 'w') as f:
                    f.write(table_text)

                # Add table information to items (format as a string for embeddings)
                table_item = {
                    "page": page_num,
                    "type": "table",
                    "text": table_text,  # Use the formatted table text here
                    "path": table_file_name
                }
                items.append(table_item)

                # Optionally remove table text from the page's text to avoid duplication
                text = text.replace(table_text, "")
    except Exception as e:
        print(f"Error extracting tables from page {page_num}: {str(e)}")
            
    # Step 2: Get all TEXT chunks in the curremt page and store 
    chunks = text_splitter.split_text(text)
    
    # Generate an item to add to items
    for i,chunk in enumerate(chunks):
        text_file_name = f"{text_save_dir}/{filename}_text_{page_num}_{i}.txt"
        # If the text folder doesn't exist, create one
        os.makedirs(text_save_dir, exist_ok=True)
        with open(text_file_name, 'w') as f:
            f.write(chunk)
        
        item={}
        item["page"] = page_num
        item["type"] = "text"
        item["text"] = chunk
        item["path"] = text_file_name
        items.append(item)
    
    
    # Step 3: Get all the IMAGES in the current page and store
    images = page.get_images()
    for idx, image in enumerate(images):        
        # Extract the image data
        xref = image[0]
        pix = pymupdf.Pixmap(doc, xref)
        pix.tobytes("png")
        # Create the image_name that includes the image path
        image_name = f"{image_save_dir}/{filename}_image_{page_num}_{idx}_{xref}.png"
        # If the image folder doesn't exist, create one
        os.makedirs(image_save_dir, exist_ok=True)
        # Save the image
        pix.save(image_name)
        
        # Produce base64 string
        with open(image_name, 'rb') as f:
            image = base64.b64encode(f.read()).decode('utf8')
        
        item={}
        item["page"] = page_num
        item["type"] = "image"
        item["path"] = image_name
        item["image"] = image
        items.append(item)

    # Step 4: Save each pdf pages as images
    pix = page.get_pixmap()
    page_path = os.path.join(page_images_save_dir, f"page_{page_num:03d}.png")
    pix.save(page_path)
    
    # Produce base64 string
    with open(page_path, 'rb') as f:
        page_image = base64.b64encode(f.read()).decode('utf8')
    
    item = {}
    item["page"] = page_num
    item["type"] = "page"
    item["path"] = page_path
    item["image"] = page_image
    items.append(item)

In [ ]:
# Looking at the first text item
[i for i in items if i['type'] == 'text'][0]

In [ ]:
# Looking at the first table item
[i for i in items if i['type'] == 'table'][0]

In [ ]:
# Looking at the first image item
[i for i in items if i['type'] == 'image'][0]

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Generating Multimodal Embeddings
</h2>

In [ ]:
import boto3 

# Generating Multimodal Embeddings using Amazon Titan Multimodal Embeddings model
def generate_multimodal_embeddings(prompt=None, image=None, output_embedding_length=384):
    """
    Invoke the Amazon Titan Multimodal Embeddings model using Amazon Bedrock runtime.

    Args:
        prompt (str): The text prompt to provide to the model.
        image (str): A base64-encoded image data.
    Returns:
        str: The model's response embedding.
    """
    if not prompt and not image:
        raise ValueError("Please provide either a text prompt, base64 image, or both as input")
    
    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(service_name="bedrock-runtime")
    model_id = "amazon.titan-embed-image-v1"
    
    body = {"embeddingConfig": {"outputEmbeddingLength": output_embedding_length}}
    
    if prompt:
        body["inputText"] = prompt
    if image:
        body["inputImage"] = image

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(body),
            accept="application/json",
            contentType="application/json"
        )

        # Process and return the response
        result = json.loads(response.get("body").read())
        return result.get("embedding")

    except ClientError as err:
        print(f"Couldn't invoke Titan embedding model. Error: {err.response['Error']['Message']}")
        return None

In [ ]:
# Set embedding vector dimension
embedding_vector_dimension = 384

# Count the number of each type of item
item_counts = {
    'text': sum(1 for item in items if item['type'] == 'text'),
    'table': sum(1 for item in items if item['type'] == 'table'),
    'image': sum(1 for item in items if item['type'] == 'image'),
    'page': sum(1 for item in items if item['type'] == 'page')
}

# Initialize counters
counters = dict.fromkeys(item_counts.keys(), 0)

# Generate embeddings for all items
with tqdm(total=len(items), desc="Generating embeddings", bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}{postfix}]") as pbar:
    for item in items:
        item_type = item['type']
        counters[item_type] += 1
        
        if item_type in ['text', 'table']:
            # For text or table, use the formatted text representation
            item['embedding'] = generate_multimodal_embeddings(prompt=item['text'], output_embedding_length=embedding_vector_dimension)
        else:
            # For images, use the base64-encoded image data
            item['embedding'] = generate_multimodal_embeddings(image=item['image'], output_embedding_length=embedding_vector_dimension)
        
        # Update the progress bar
        pbar.set_postfix_str(f"Text: {counters['text']}/{item_counts['text']}, Table: {counters['table']}/{item_counts['table']}, Image: {counters['image']}/{item_counts['image']}")
        pbar.update(1)

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Creating Vector Database/Index
</h2>

![Vector Database](data/vectordb.png)


In [ ]:
# All the embeddings
all_embeddings = np.array([item['embedding'] for item in items])

# Create FAISS Index
index = faiss.IndexFlatL2(embedding_vector_dimension)

# Clear any pre-existing index
index.reset()

# Add embeddings to the index
index.add(np.array(all_embeddings, dtype=np.float32))

In [ ]:
# Generating RAG response with Claude 3.5/3
def invoke_claude_3_multimodal(prompt, images, image_types):
    """
    Invoke the Claude-3 multimodal model from Anthropic using AWS Bedrock runtime.

    Args:
        prompt (str): The text prompt to provide to the model.
        images (list): A list of base64-encoded image data.
        image_types (list): A list of MIME types corresponding to the images.

    Returns:
        str: The model's response text.

    Raises:
        ValueError: If an invalid model name is provided.
    """
    # Initialize the Amazon Bedrock runtime client and model ID 
    client = boto3.client(service_name="bedrock-runtime")
    # model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"     
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"        # Replace with the model ID you want to use

    # Prepare the multimodal prompt message
    message_content = []

    # Add each image to the message content
    for image, img_type in zip(images, image_types):
        message_content.append({
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": img_type,
                "data": image,
            },
        })
    message_content.append({"type": "text", "text": prompt})

    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1.0,
        "top_k": 250,
        "messages": [
            {
                "role": "user",
                "content": message_content,
            }
        ],
    }

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(request_body),
        )

        # Process and return the response
        result = json.loads(response.get("body").read())
        return result['content'][0]['text']

    except ClientError as err:
        logger.error(
            "Couldn't invoke Claude 3 %s model. Here's why: %s: %s",
            model_id.split('.')[-1].capitalize(),
            err.response["Error"]["Code"],
            err.response["Error"]["Message"],
        )
        raise

In [ ]:
# Generate RAG response with Claude 3.5/3
def generate_rag_response(prompt, matched_items):
    
    # Create context
    text_context = ""
    image_context = []
    
    for item in matched_items:
        if item['type'] == 'text' or item['type'] == 'table':
            text_context += str(item["page"]) + ". " + item['text'] + "\n"
        else:
            image_context.append(item['image'])
    
    # Only 5 images are supported by Claude3 models
    if len(image_context) > 5:
        image_context = image_context[:5]
    
    final_prompt = f"""You are a helpful assistant for question answering.
    The text context is relevant information retrieved.
    The provided image(s) are relevant information retrieved.
    
    <context>
    {text_context}
    </context>
    
    Answer the following question using the relevant context and images.
    
    <question>
    {prompt}
    </question>
    
    Answer:"""

    response = invoke_claude_3_multimodal(final_prompt, image_context, ['image/png' for _ in image_context])
    
    return response
    

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Test the RAG Pipeline
</h2>

In [ ]:
# User Query
query = "How is the scaled-dot-product attention is calculated?"

# Generate embeddings for the query
query_embedding = generate_multimodal_embeddings(prompt=query,output_embedding_length=embedding_vector_dimension)

# Search for the nearest neighbors in the vector database
distances, result = index.search(np.array(query_embedding, dtype=np.float32).reshape(1,-1), k=5)

In [ ]:
# Check the result (matched chunks)
result.flatten()

In [ ]:
# Retrieve the matched items
matched_items = [items[index] for index in result.flatten()]

# Generate a response using the RAG pipeline
response = generate_rag_response(query, matched_items)

# Display the response
display.Markdown(response)

In [ ]:
# Query
query = "What is the BLEU score of the model in English to German translation (EN-DE)?"

# Generate embeddings for the query
query_embedding = generate_multimodal_embeddings(prompt=query,output_embedding_length=embedding_vector_dimension)

# Search for the nearest neighbors in the vector database
distances, result = index.search(np.array(query_embedding, dtype=np.float32).reshape(1,-1), k=5)
result.flatten()

# Retrieve the matched items
matched_items = [items[index] for index in result.flatten()]

# Generate a response using the RAG pipeline
response = generate_rag_response(query, matched_items)

# Display the response
display.Markdown(response)

<h2 style="background: linear-gradient(to right, #ff6b6b, #4ecdc4, #1e90ff); 
            color: white; 
            padding: 15px; 
            border-radius: 10px; 
            text-align: center; 
            font-family: 'Comic Sans MS', cursive, sans-serif; 
            text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
  Your Turn: Test the RAG Pipeline
</h2>

In [ ]:
# List of queries (Replace with any query of your choice)
other_queries = ["How long were the base and big models trained?",
                 "Which optimizer was used when training the models?",
                 "What is the position-wise feed-forward neural network mentioned in the paper?",
                 "What is the BLEU score of the model in English to French translation (EN-FR)?",
                 "What is the BLEU score of the model in English to German translation (EN-DE)?",
                 "How is the scaled-dot-product attention is calculated?",
                 "What is the BLEU score of the model in English to French translation (EN-FR)?",
                 "What is the BLEU score of the model in English to German translation (EN-DE)?",
                 "How is the scaled-dot-product attention is calculated?",
                 ]


In [ ]:
query = other_queries[0] # Replace with any query from the list above

# Generate embeddings for the query
query_embedding = generate_multimodal_embeddings(prompt=query,output_embedding_length=embedding_vector_dimension)

# Search for the nearest neighbors in the vector database
distances, result = index.search(np.array(query_embedding, dtype=np.float32).reshape(1,-1), k=5)
result.flatten()

# Retrieve the matched items
matched_items = [items[index] for index in result.flatten()]

# Generate a response using the RAG pipeline
response = generate_rag_response(query, matched_items)

# Display the response
display.Markdown(response)